# ***Hedef Degisken ve Ozellikler Arasindaki Iliski***

***Ev fiyatları modeli***

Ev fiyatları modelinizi tekrar çalıştırın ve sonuçları yorumlayın. 

- Hangi özellikler istatistiksel olarak anlamlı ve hangileri değildir?

- Şimdi, anlamsız özellikleri modelinizden hariç tutun. Bir şey değişti mi?

- Ev fiyatları ile olan ilişkilerini belirleyerek istatistiksel olarak anlamlı katsayıları yorumlayın. 

- Ev fiyatları üzerinde hangi özellikler daha belirgin bir etkiye sahiptir?

- Sonuçlar size mantıklı geliyor mu? Değilse, potansiyel nedenleri açıklamaya çalışın.

In [1]:
import pandas as pd
import numpy as np
# grafiksel araclar
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
# lineer regresyon icin
from sklearn import linear_model
import statsmodels.api as sm
# polynomial features icin
from sklearn.preprocessing import PolynomialFeatures

# path ile calismak icin
import sys
# dosyanin bulundugu dizini path e ekledik
mylib = '/home/admin1/datascience/datascience/library'
if mylib not in sys.path:
    sys.path.insert(1, mylib)
# parser i kullanabilmek icin
from data_description_parser import data_description_house_prices
data_dict = data_description_house_prices()

# uyarilari dikkate alma
import warnings
warnings.filterwarnings('ignore')

# pandas varsayilan olarak cok sayida sutun veya satir varsa tumunu gostermez
# bu nedenle 100 sutun ve satir gostermesi icin
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

# virgulden sonra belirli sayida basamak gosterimi icin 
pd.options.display.float_format = '{:,.3f}'.format

# font tanimlamalari
title_font = {'family': 'times new roman', 'color': 'darkred','weight': 'bold','size': 14}
axis_font  = {'family': 'times new roman', 'color': 'darkred','weight': 'bold','size': 14}

# temiz veri setini dataframe icerisine yukle
house_prices = pd.read_csv('data/house_prices_out.csv',index_col='Id')

house_prices

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.000,8450,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.000,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,None,Attchd,"2,003.000",RFn,2,548,TA,TA,Y,0,61,0,0,0,0,None,None,None,0,2,2008,WD,Normal,208500
2,20,RL,80.000,9600,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.000,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,"1,976.000",RFn,2,460,TA,TA,Y,298,0,0,0,0,0,None,None,None,0,5,2007,WD,Normal,181500
3,60,RL,68.000,11250,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.000,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,"2,001.000",RFn,2,608,TA,TA,Y,0,42,0,0,0,0,None,None,None,0,9,2008,WD,Normal,223500
4,70,RL,60.000,9550,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.000,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,"1,998.000",Unf,3,642,TA,TA,Y,0,35,272,0,0,0,None,None,None,0,2,2006,WD,Abnorml,140000
5,60,RL,84.000,14260,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.000,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,"2,000.000",RFn,3,836,TA,TA,Y,192,84,0,0,0,0,None,None,None,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,62.000,7917,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,None,0.000,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,Ex,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,"1,999.000",RFn,2,460,TA,TA,Y,0,40,0,0,0,0,None,None,None,0,8,2007,WD,Normal,175000
1457,20,RL,85.000,13175,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.000,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,TA,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,"1,978.000",Unf,2,500,TA,TA,Y,349,0,0,0,0,0,None,MnPrv,None,0,2,2010,WD,Normal,210000
1458,70,RL,66.000,9042,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,None,0.000,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,Ex,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,"1,941.000",RFn,1,252,TA,TA,Y,0,60,0,0,0,0,None,GdPrv,Shed,2500,5,2010,WD,Normal,266500


***Kategorik Bilesenleri Encode Etme***

- Kategorik degiskenleri encode ederken groupby metodu ile degiskenin degerlerinin SalePrice ortalamalarina gore degerler 1 den baslayarak siralandi(puanlandi).
- Mesela ExterQual icin: {'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}

In [116]:
# object tipinde olanlar kategorik degiskenler listesine ayir
qualitative = [column for column in house_prices.columns if house_prices.dtypes[column] == 'object']

# object tipinde olanlar kategorik degiskenler listesine ayir
quantitative = [column for column in house_prices.columns if house_prices.dtypes[column] != 'object']

len(qualitative)
len(quantitative)

37

In [117]:
def encode(df, feature):
    ordering = pd.DataFrame()
    ordering['val'] = df[feature].unique()
    # degerler index yapildi
    ordering.index = ordering.val
    # degisken degerlerinin SalePrice ortalamalarini aliyor
    ordering['spmean'] = df[[feature, 'SalePrice']].groupby(feature).mean()['SalePrice']
    # ortalamalari siraliyor
    ordering = ordering.sort_values('spmean')
    # siralama degerleri ayri bir sutunda
    ordering['ordering'] = range(1, ordering.shape[0]+1)
    #print(ordering)
    ordering = ordering['ordering'].to_dict()    
    # en son soyle bir sozluk olusur ExterQual icin
    # {'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}
    
    # burasi cok ilginc yeni encoded degiskeni burada olusturuyor
    for cat, o in ordering.items():        
        df.loc[df[feature] == cat, feature+'_E'] = o
        
qual_encoded = []
for q in qualitative:  
    encode(house_prices, q)
    qual_encoded.append(q+'_E')

In [118]:
quantitative.remove('SalePrice')

In [119]:
expl_vars = quantitative + qual_encoded
# hedef degiskeni
Y = house_prices['SalePrice']
# aciklayici degiskenler
X = house_prices[expl_vars]

In [120]:
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.902
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     162.4
Date:                Sat, 11 Apr 2020   Prob (F-statistic):               0.00
Time:                        16:56:08   Log-Likelihood:                -16532.
No. Observations:                1442   AIC:                         3.322e+04
Df Residuals:                    1364   BIC:                         3.363e+04
Df Model:                          77                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const           -1.353e+06   1.01e+06     -1.341      0.180   -3.33e+06    6.26e+05
MSSubClass       -133.5363     31.179     -4.283      0.000    -194.700     -72.372
LotFrontage        63.0816     21.405      2.947      0.003      21.091     105.072
LotArea             0.3927      0.083      4.726      0.000       0.230       0.556
OverallQual      8994.3764    933.879      9.631      0.000    7162.381    1.08e+04
OverallCond      4701.1929    790.513      5.947      0.000    3150.439    6251.947
YearBuilt          98.6721     61.022      1.617      0.106     -21.035     218.379
YearRemodAdd      -10.1130     53.160     -0.190      0.849    -114.397      94.171
MasVnrArea         22.1482      4.929      4.494      0.000      12.480      31.817
BsmtFinSF1         16.1792      2.067      7.829      0.000      12.125      20.233
BsmtFinSF2          1.4380      3.619      0.397      0.691      -5.662       8.538
BsmtUnfSF          -3.8610      2.092     -1.845      0.065      -7.966       0.244
TotalBsmtSF        13.7561      2.981      4.614      0.000       7.908      19.604
1stFlrSF           27.2647      4.732      5.761      0.000      17.981      36.548
2ndFlrSF           25.0528      4.336      5.778      0.000      16.547      33.559
LowQualFinSF      -24.9745     11.049     -2.260      0.024     -46.649      -3.300
GrLivArea          27.3431      4.285      6.380      0.000      18.936      35.750
BsmtFullBath      858.3052   1904.043      0.451      0.652   -2876.865    4593.476
BsmtHalfBath    -3535.3107   2918.759     -1.211      0.226   -9261.055    2190.433
FullBath        -1455.5958   2023.965     -0.719      0.472   -5426.018    2514.826
HalfBath         2100.8426   1926.706      1.090      0.276   -1678.787    5880.472
BedroomAbvGr    -5909.5570   1262.648     -4.680      0.000   -8386.499   -3432.615
KitchenAbvGr    -2.176e+04   4720.022     -4.611      0.000    -3.1e+04   -1.25e+04
TotRmsAbvGrd     2945.1642    893.822      3.295      0.001    1191.749    4698.579
Fireplaces       1627.4157   1874.293      0.868      0.385   -2049.393    5304.224
GarageYrBlt       -16.7141     48.064     -0.348      0.728    -111.001      77.573
GarageCars       2380.2712   2196.660      1.084      0.279   -1928.928    6689.470
GarageArea         22.8478      7.304      3.128      0.002       8.519      37.177
WoodDeckSF         17.3108      5.732      3.020      0.003       6.067      28.555
OpenPorchSF        22.6136     10.856      2.083      0.037       1.318      43.909
EnclosedPorch       5.9945     12.103      0.495      0.620     -17.747      29.737
3SsnPorch          12.5977     21.628      0.582      0.560     -29.830      55.025
ScreenPorch        25.7568     12.267      2.100      0.036       1.692      49.821
PoolArea           46.3581     43.575      1.064      0.288

***Istatistiksel olarak anlamli olmayan degiskenler***
- p degeri yuksek olan degiskenler (```pval>0.005```)

In [123]:
len(expl_vars)

79

In [127]:
vars_to_remove = []
# p degerleri yuksek olanlari cikar
for var in expl_vars:
    if results.pvalues[var]>0.05:
        vars_to_remove.append(var)

In [128]:
len(vars_to_remove)

45

In [59]:
data_dict = data_description_house_prices()

In [130]:
# istenmeyenleri ayir
for var in vars_to_remove:
    if var in expl_vars:
        expl_vars.remove(var)

In [131]:
len(expl_vars)

34

***Degiskenler azaltildiktan sonra modelin olusturulmasi***

In [132]:
# hedef degiskeni
Y = house_prices['SalePrice']
# aciklayici degiskenler
X = house_prices[expl_vars]

X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.895
Method:                 Least Squares   F-statistic:                     371.8
Date:                Sat, 11 Apr 2020   Prob (F-statistic):               0.00
Time:                        16:58:32   Log-Likelihood:                -16565.
No. Observations:                1442   AIC:                         3.320e+04
Df Residuals:                    1408   BIC:                         3.338e+04
Df Model:                          33                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const           -1.883e+05   2.43e+04     -7.759      0.000   -2.36e+05   -1.41e+05
MSSubClass        -96.8205     19.256     -5.028      0.000    -134.595     -59.046
LotFrontage        54.1393     20.054      2.700      0.007      14.801      93.478
LotArea             0.4318      0.072      5.963      0.000       0.290       0.574
OverallQual      9153.4515    897.597     10.198      0.000    7392.681    1.09e+04
OverallCond      4067.0896    651.599      6.242      0.000    2788.880    5345.299
MasVnrArea         21.5791      4.251      5.077      0.000      13.241      29.918
BsmtFinSF1         20.5506      1.784     11.523      0.000      17.052      24.049
TotalBsmtSF        10.6993      3.615      2.959      0.003       3.607      17.791
1stFlrSF           27.6625      4.401      6.286      0.000      19.030      36.295
2ndFlrSF           24.9984      3.940      6.344      0.000      17.269      32.728
LowQualFinSF      -24.9443     10.492     -2.378      0.018     -45.525      -4.363
GrLivArea          27.7167      3.942      7.031      0.000      19.983      35.450
BedroomAbvGr    -6460.8107   1183.674     -5.458      0.000   -8782.765   -4138.857
KitchenAbvGr    -1.531e+04   3656.313     -4.188      0.000   -2.25e+04   -8140.188
TotRmsAbvGrd     2886.0768    871.651      3.311      0.001    1176.202    4595.952
GarageArea         20.3379      4.124      4.932      0.000      12.248      28.428
WoodDeckSF         18.5418      5.542      3.346      0.001       7.671      29.413
OpenPorchSF        20.3576     10.543      1.931      0.054      -0.323      41.039
ScreenPorch        25.7825     11.820      2.181      0.029       2.596      48.969
Street_E         1.839e+04   1.02e+04      1.794      0.073   -1717.056    3.85e+04
Neighborhood_E   1303.5110    155.081      8.405      0.000     999.295    1607.727
Condition1_E     2157.7466    660.894      3.265      0.001     861.303    3454.190
RoofStyle_E      2388.2771    832.842      2.868      0.004     754.531    4022.023
Exterior1st_E    1758.1337    519.265      3.386      0.001     739.518    2776.750
Exterior2nd_E    -851.4616    408.462     -2.085      0.037   -1652.721     -50.202
ExterQual_E      1.105e+04   1914.663      5.771      0.000    7293.758    1.48e+04
BsmtQual_E       4571.3114   1492.572      3.063      0.002    1643.407    7499.215
BsmtCond_E      -6923.6114   1975.769     -3.504      0.000   -1.08e+04   -3047.844
BsmtExposure_E   4391.3368    715.510      6.137      0.000    2987.756    5794.918
Electrical_E    -2806.8826   1771.342     -1.585      0.113   -6281.637     667.871
KitchenQual_E    7067.7485   1499.024      4.715      0.000    4127.187       1e+04
Functional_E     5165.3562    907.094      5.694      0.000    3385.955    6944.758
SaleType_E       5002.9534    652.097      7.672      0.000

- Anlamli olmayan 45 degisken cikarildiktan sonra R kare degeri %90.2 den %89.7 ye dustu yani modelimin varyansi ifade etme yuzdesi degismedi.

- 'OverallQual', 'OverallCond', 'TotRmsAbvGrd', 'Neighborhood_E', 'Condition1_E', 'RoofStyle_E', 'Exterior1st_E', 'BsmtQual_E', 'KitchenQual_E', 'Functional_E', 'SaleType_E', 'SaleCondition_E' degiskenlerinin etkisi digerlerine gore daha fazla.

- Hala anlamsiz degiskenler mevcut : 'OpenPorchSF', 'Electrical_E'

In [133]:
expl_vars.remove('OpenPorchSF')
expl_vars.remove('Electrical_E')

***Etkilesimli degiskenler***

- Scikit learn kutuphanesinden PolynomialFeatures  modulu kullanilacak.

In [134]:
# hedef degiskeni
Y = house_prices['SalePrice']
# aciklayici degiskenler
X = house_prices[expl_vars]


# model nesnesi olusturuldu
lrm = linear_model.LinearRegression()
# modelimizin denklemi (egrisi) olusturuluyor
lrm.fit(X, Y)

# degiskenler ve sabit(bias) degeri
print('Değişkenler: \n', lrm.coef_)
print('Sabit değer (bias): \n', lrm.intercept_)

Değişkenler: 
 [-1.00330606e+02  5.33963352e+01  4.32076206e-01  9.20922670e+03
  3.98542918e+03  2.12774004e+01  2.03554517e+01  1.15360116e+01
  2.75885162e+01  2.58470480e+01 -2.54437089e+01  2.79918553e+01
 -6.67101818e+03 -1.49759596e+04  2.86654749e+03  2.00892460e+01
  1.76492192e+01  2.64805224e+01  1.75149747e+04  1.26845976e+03
  2.14415779e+03  2.37871523e+03  1.73522072e+03 -8.51521879e+02
  1.13568544e+04  4.62078166e+03 -7.53446155e+03  4.36909537e+03
  6.91545873e+03  5.13974591e+03  5.11199884e+03  2.85051161e+03]
Sabit değer (bias): 
 -196882.7681915523


In [135]:
# kacinci dereceden polinomlar olusturacagiz belirtiyoruz
poly = PolynomialFeatures(2)
X = poly.fit_transform(X)

X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.961
Model:                            OLS   Adj. R-squared:                  0.941
Method:                 Least Squares   F-statistic:                     47.29
Date:                Sat, 11 Apr 2020   Prob (F-statistic):               0.00
Time:                        16:59:18   Log-Likelihood:                -15862.
No. Observations:                1442   AIC:                         3.272e+04
Df Residuals:                     946   BIC:                         3.533e+04
Df Model:                         495                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.477e+04   9213.606      2.689      0.007    6691.244    4.29e+04
x1           149.8706    237.667      0.631      0.528    -316.544     616.285
x2           194.1589    297.292      0.653      0.514    -389.269     777.586
x3            -0.5472      5.418     -0.101      0.920     -11.180      10.086
x4         -6984.8903   3979.651     -1.755      0.080   -1.48e+04     825.074
x5          1050.6304   2987.056      0.352      0.725   -4811.392    6912.653
x6          -127.2319    156.461     -0.813      0.416    -434.282     179.818
x7             5.1138    181.253      0.028      0.977    -350.590     360.818
x8           188.3874    234.211      0.804      0.421    -271.245     648.020
x9          -249.8250    108.534     -2.302      0.022    -462.821     -36.829
x10          -18.9666     39.178     -0.484      0.628     -95.852      57.919
x11           59.5955    108.849      0.548      0.584    -154.017     273.208
x12         -209.1982    105.657     -1.980      0.048    -416.548      -1.848
x13         4953.4644   5321.546      0.931      0.352   -5489.936    1.54e+04
x14        -3.512e+04   2.22e+04     -1.585      0.113   -7.86e+04    8352.429
x15         9047.7772   3811.303      2.374      0.018    1568.191    1.65e+04
x16          339.5301    196.678      1.726      0.085     -46.445     725.506
x17         -142.4612    285.710     -0.499      0.618    -703.160     418.238
x18         -504.0335    654.556     -0.770      0.441   -1788.583     780.516
x19         4.954e+04   1.84e+04      2.688      0.007    1.34e+04    8.57e+04
x20          -61.9195    769.911     -0.080      0.936   -1572.851    1449.012
x21        -1946.5762   2947.851     -0.660      0.509   -7731.660    3838.508
x22         -156.5441   3933.975     -0.040      0.968   -7876.870    7563.782
x23        -1378.5595   3085.681     -0.447      0.655   -7434.132    4677.013
x24          423.6023   2468.453      0.172      0.864   -4420.674    5267.879
x25        -7351.7392   1.05e+04     -0.698      0.485    -2.8e+04    1.33e+04
x26         3201.8149   6822.652      0.469      0.639   -1.02e+04    1.66e+04
x27        -1.096e+04   6790.574     -1.614      0.107   -2.43e+04    2363.481
x28         -910.5984   3729.484     -0.244      0.807   -8229.616    6408.420
x29         -168.8078   6782.801     -0.025      0.980   -1.35e+04    1.31e+04
x30        -9171.3886   4750.797     -1.930      0.054   -1.85e+04     151.931
x31        -6809.5030   4918.988     -1.384      0.167   -1.65e+04    2843.887
x32          -56.5174   4551.432     -0.012      0.990   -8988.589    8875.554
x33            0.6562      0.678      0.968      0.333      -0.675       1.987
x34           -0.1876      0.994     -0.189      0.850      -2.138       1.763
x35           -0.0039      0.006     -0.615      0.539      -0.016       0.008
x3

- 560 adet 2 inci dereceden polynomial degisken tespit etti ve modelimin R kare degeri %89 dan % 96 ya yukseldi.